In [6]:
import calendar
from datetime import datetime, date, timedelta


In [25]:
def get_fourth_wednesday(year, month):
    """
    返回指定年月的第 4 个星期三的日期。
    
    参数:
    - year: 年份 (int)
    - month: 月份 (int)
    
    返回:
    - 第 4 个星期三的日期 (datetime.date)
    """
    # 每月的第一天是星期几（0=星期一，6=星期日）以及该月的天数
    first_day_of_month_weekday, days_in_month = calendar.monthrange(year, month)
    
    # 计算第一个星期三的日期
    # 如果第一天是星期四(3)或更晚，那么第一个星期三会在第二周
    first_wednesday = (2 - first_day_of_month_weekday + 7) % 7 + 1
    
    # 计算第四个星期三的日期
    fourth_wednesday = first_wednesday + 3 * 7  # 在第一个星期三的基础上加上 3 周
    
    # 如果计算结果超出了月份的天数，说明第一个星期三在第一周，需要调整
    if fourth_wednesday > days_in_month:
        fourth_wednesday -= 7
    
    return datetime(year, month, fourth_wednesday).date()

def get_exit_date(dt, month_type, expired_date):
    (year, month) = (dt.year, dt.month + 1) if dt.month < 12 else (dt.year+1, 1)
    next_expired_date = get_fourth_wednesday(year, month)
    
    if month_type == 1 and dt.date() >= expired_date:            
        exit_date = next_expired_date
    elif month_type == 0 and dt.date() < expired_date:
        exit_date = expired_date
    else:
        diff_days = (expired_date - dt.date()).days
        exit_date = next_expired_date - timedelta(days=diff_days)
        
    return exit_date.strftime('%Y-%m-%d')

In [27]:
get_exit_date(datetime(2020, 10, 29), 0,  date(2020, 10, 28))

'2020-11-26'